## Lab 5: AgentCore Evaluations - カスタマーサポートエージェントのオンライン評価

### 概要

このLabでは、AgentCore Evaluations を使用して、Lab 4 の本番カスタマーサポートエージェントを継続的に監視する方法を示します。顧客がエージェントとやり取りする際にリアルタイムでエージェントのパフォーマンスを自動的に評価するオンライン評価を設定します。

**ワークショップの流れ:**

- **Lab 1（完了）:** エージェントプロトタイプの作成 - 機能するカスタマーサポートエージェントを構築
- **Lab 2（完了）:** Memory による強化 - 会話コンテキストとパーソナライゼーションを追加
- **Lab 3（完了）:** Gateway と Identity によるスケーリング - エージェント間でツールを安全に共有
- **Lab 4（完了）:** 本番環境へのデプロイ - AgentCore Runtime を使用してオブザーバビリティを実現
- **Lab 5（現在）:** エージェントパフォーマンスの評価 - オンライン評価で品質を監視
- **Lab 6:** ユーザーインターフェースの構築 - 顧客向けアプリケーションを作成

### 学習内容

組み込み評価器でオンライン評価を設定し、テストインタラクションを生成し、AgentCore Observability ダッシュボードを通じて品質メトリクスを分析して、エージェントのパフォーマンスを向上させます。

### オンライン評価の概要

オンライン評価は、選択した特定のインタラクションを分析するオンデマンド評価とは異なり、本番環境でデプロイされたエージェントを継続的に監視します。3つのコンポーネントで構成されます：設定可能なルールを持つセッションサンプリング、複数の評価方法（組み込みまたはカスタム評価器）、品質トレンドと低スコアセッション調査を備えたダッシュボードによる監視。

エージェントは AgentCore Runtime で実行されているため、AgentCore Observability は [OTEL](https://opentelemetry.io/) インストルメンテーションを使用して自動的にコードをインストルメント化し、包括的なログとトレースを提供します。

### 前提条件

Lab 4 を完了して、カスタマーサポートエージェントがデプロイされている必要があります。Evaluations 権限を持つ Amazon Bedrock AgentCore への AWS アカウントアクセスが必要です。

### アーキテクチャ
<div style="text-align:left">
    <img src="images/architecture_lab5_evaluation.png" width="75%"/>
</div>

*オンライン評価は、エージェントのインタラクションを自動的に監視し、サンプリングルールに基づいて評価器を適用し、結果を分析のために CloudWatch に出力します。*

### ステップ 1: 必要なライブラリのインポートとクライアントの初期化

In [ ]:
!pip install -r requirements.txt

In [ ]:
from bedrock_agentcore_starter_toolkit import Evaluation, Runtime
import json
import uuid
from pathlib import Path
from boto3.session import Session
from IPython.display import Markdown, display
from lab_helpers.utils import get_ssm_parameter, get_or_create_cognito_pool

In [ ]:
boto_session = Session()
region = boto_session.region_name
print(f"リージョン: {region}")

In [ ]:
eval_client = Evaluation(region=region)
runtime_client = Runtime()

### ステップ 2: Lab 4 からのエージェント情報の取得

Lab 4 のデプロイ中に SSM Parameter Store に保存されたカスタマーサポートエージェント ARN を取得します。

In [ ]:
try:
    # Get agent ARN from SSM parameter store (saved in Lab 4)
    agent_arn = get_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
    
    # Extract agent ID from ARN
    agent_id = agent_arn.split(":")[-1].split("/")[-1]
    
    # Set runtime client config path
    runtime_client._config_path = Path.cwd() / ".bedrock_agentcore.yaml"
    
    print("エージェント ID:", agent_id)
    print("エージェント ARN:", agent_arn)
except Exception as e:
    raise Exception(f"""Lab 4 からのエージェント情報がありません。最初に lab-04-agentcore-runtime.ipynb を実行してください。エラー: {str(e)}""")

### ステップ 3: オンライン評価設定の作成

カスタマーサポートエージェント用のオンライン評価設定を作成しましょう。組み込み評価器を使用して、エージェントパフォーマンスのさまざまな側面を評価します：

- **Builtin.GoalSuccessRate** - エージェントがユーザーの目標をどの程度達成しているかを測定
- **Builtin.Correctness** - レスポンスの事実の正確性を評価
- **Builtin.ToolSelectionAccuracy** - 適切なツール選択を評価

デモンストレーションのためにサンプリングレートを100%に設定していますが、本番環境ではトラフィック量に基づいてより低いレート（例：10〜20%）を使用する場合があります。

In [ ]:
response = eval_client.create_online_config(
    agent_id=agent_id,
    config_name="customer_support_agent_eval",
    sampling_rate=100,  # Evaluate 100% of sessions for demo
    evaluator_list=[
        "Builtin.GoalSuccessRate", 
        "Builtin.Correctness",
        "Builtin.ToolSelectionAccuracy"
    ],
    config_description="Customer support agent online evaluation",
    auto_create_execution_role=True
)

print("オンライン評価設定が正常に作成されました！")
print(f"設定 ID: {response['onlineEvaluationConfigId']}")

### ステップ 4: 設定ステータスの確認

詳細を取得して、評価設定が適切に作成され有効になっていることを確認します。

In [ ]:
config_details = eval_client.get_online_config(config_id=response['onlineEvaluationConfigId'])
print("設定詳細:")
print(json.dumps(config_details, indent=2, default=str))

### ステップ 5: テストインタラクションの生成

さまざまなクエリでカスタマーサポートエージェントを呼び出して、評価用のトレースを生成します。異なるテストシナリオにより、評価器がエージェントのパフォーマンスをどのように評価するかを示します。

In [ ]:
# Get authentication token
access_token = get_or_create_cognito_pool(refresh_token=True)
print(f"アクセストークンを取得しました: {access_token['bearer_token'][:20]}...")

def invoke_agent_runtime(prompt, session_id=None):
    """Invoke the agent runtime using starter toolkit"""
    if not session_id:
        session_id = str(uuid.uuid4())
    
    response = runtime_client.invoke(
        payload={"prompt": prompt},
        session_id=session_id,
        bearer_token=access_token['bearer_token']
    )
    
    return response, session_id

#### テストシナリオ 1: 製品情報クエリ

In [ ]:
session1 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "I need information about the Gaming Console Pro. What are its specifications and price?",
    session1
)
print("顧客クエリ: 製品情報リクエスト")
display(Markdown(response["response"].replace('\\n', '\n')))

#### テストシナリオ 2: テクニカルサポートリクエスト

In [ ]:
session2 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "My laptop won't start up. Can you help me troubleshoot this issue?",
    session2
)
print("顧客クエリ: テクニカルサポートリクエスト")
display(Markdown(response["response"].replace('\\n', '\n')))

#### テストシナリオ 3: 返品ポリシーの問い合わせ

In [ ]:
session3 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "I bought a smartphone last week but it's not working properly. What's your return policy?",
    session3
)
print("顧客クエリ: 返品ポリシーの問い合わせ")
display(Markdown(response["response"].replace('\\n', '\n')))

#### テストシナリオ 4: 複雑なマルチツールクエリ

In [ ]:
session4 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "I need help with my Gaming Console Pro. First, can you tell me about its warranty? Then I need technical support for connection issues.",
    session4
)
print("顧客クエリ: 複雑なマルチツールリクエスト")
display(Markdown(response["response"].replace('\\n', '\n')))

#### テストシナリオ 5: 一般的な機能クエリ

In [ ]:
session5 = str(uuid.uuid4())
response, _ = invoke_agent_runtime(
    "What kind of support can you provide? List all your available tools and capabilities.",
    session5
)
print("顧客クエリ: 機能の問い合わせ")
display(Markdown(response["response"].replace('\\n', '\n')))

### ステップ 6: 評価結果の監視

AgentCore Observability コンソールで評価結果を監視します。システムがトレースを処理し評価器を適用するため、結果が表示されるまで数分かかる場合があります。

#### ダッシュボードへのアクセス

1. [AgentCore Observability コンソール](https://console.aws.amazon.com/cloudwatch/home#gen-ai-observability/agent-core/agents) に移動します
2. エージェントリストでカスタマーサポートエージェントを見つけます
3. `DEFAULT` エンドポイントをクリックして評価メトリクスを表示します
4. トレースとセッションビューで評価スコアを探します

#### 表示される内容

ダッシュボードには以下が表示されます：
- **Goal Success Rate**: エージェントが顧客の目標をどの程度達成しているか
- **Correctness**: 提供された情報の正確性
- **Tool Selection Accuracy**: クエリに対する適切なツール選択

![オンライン評価ダッシュボード](images/online_evaluations_dashboard.png)

*AgentCore Observability ダッシュボードに表示される評価メトリクス*

### ステップ 7: 評価メトリクスの理解

**Goal Success Rate** は、エージェントが顧客の主要な意図に正常に対応しているかどうかを測定します。高いスコアは効果的な問題解決を示し、低いスコアは満たされていないニーズ、不完全なレスポンス、または誤解されたリクエストを示唆します。

**Correctness** は、レスポンスの事実の正確性を評価します。高いスコアは正確で信頼できる情報を示し、低いスコアは誤った事実、古い情報、または誤解を招くガイダンスを示唆します。

**Tool Selection Accuracy** は、エージェントが各タスクに適切なツールを選択しているかどうかを評価します。高いスコアは適切なツール選択を示し、低いスコアは間違ったツール、不要な呼び出し、またはツール使用の欠落を示唆します。

### ステップ 8: 結果の分析と次のステップ

**Goal Success Rate が低い場合:** エージェントのシステムプロンプトを改善し、ツールの説明とパラメータを改善し、特定のトレーニング例を追加します。

**Correctness スコアが低い場合:** Knowledge Base を最新の情報で更新し、ファクトチェックメカニズムを改善し、ツールのレスポンスを確認します。

**ツール関連の問題の場合:** ツールパラメータスキーマを改善し、ツール選択ロジックを改善し、ツールドキュメントを強化します。

**継続的な監視:** 評価メトリクス用の CloudWatch アラームを設定し、トレンド分析用のダッシュボードを作成し、品質低下の自動アラートを実装します。

### ステップ 9: クリーンアップ（オプション）

必要に応じて、以下のコードのコメントを解除してオンライン評価設定を無効にします。

In [ ]:
# Uncomment the following lines if you want to disable the evaluation configuration
# eval_client.delete_online_config(config_id=response['onlineEvaluationConfigId'])
# print("オンライン評価設定が無効化されました")

### おめでとうございます！

**Lab 5: AgentCore Evaluations - オンライン評価**を正常に完了しました！

### 達成したこと

Goal Success Rate（顧客満足度と問題解決）、Correctness（事実の正確性）、Tool Selection Accuracy（適切なツール使用）を評価する組み込み評価器で、カスタマーサポートエージェントの自動継続的オンライン評価を設定しました。評価結果は、リアルタイムのインサイトのために AgentCore Observability ダッシュボードと統合されています。

**主な利点:** 顧客への影響前に問題をキャッチするプロアクティブな品質保証、改善をガイドするデータドリブンな最適化、大規模なパフォーマンス監視による本番環境の信頼性、パターンと機会を特定する継続的な学習。

**次のステップ:** 評価ダッシュボードを定期的に監視し、品質しきい値用の CloudWatch アラームを設定し、インサイトを使用してエージェントを反復的に改善し、ドメイン固有のメトリクス用のカスタム評価器の追加を検討してください。

### 次のステップ: [Lab 6: ユーザーインターフェースの構築 →](lab-06-frontend.ipynb)

品質監視されたエージェントと顧客がやり取りするための、ユーザーフレンドリーなウェブインターフェースを構築して、顧客体験を完成させましょう。

カスタマーサポートエージェントは、包括的な品質監視を備えて本番環境対応になりました！